In [1]:
import torch
import torch.nn as nn
import tiktoken

In [2]:
tokenizer = tiktoken.get_encoding("gpt2")

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, num_heads, context_length, dropout, qkv_bias = False):
        super().__init__()
        assert d_out % num_heads == 0 
        self.d_in = d_in
        self.d_out = d_out
        self.head_dim = d_out // num_heads
        self.num_heads = num_heads
        self.context_length = context_length
        self.w_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.w_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.out_proj = nn.Linear(d_out, d_out)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(self.context_length, self.context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.w_key(x) # Applying key matrix
        values = self.w_value(x) # Applying value matrix
        queries = self.w_query(x)  # Applying query matrix

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) # Reshaping keys so we can apply attention to all the batches at once rather than splitting them and then applying attention
        values = values.view(b, num_tokens, self.num_heads, self.head_dim) # Reshaping values so we can apply attention to all the batches at once rather than splitting them and then applying attention
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim) # Reshaping queries so we can apply attention to all the batches at once rather than splitting them and then applying attention

        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        queries = queries.transpose(1,2)

        attention_scores = queries @ keys.transpose(2, 3) # Calculating attention scores
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # Applying -inf to the upper triangular matrix so softmax will zero out the values
        attention_scores = attention_scores.masked_fill_(mask_bool, -torch.inf)

        attention_weights = torch.softmax(attention_scores / (keys.shape[-1]**0.5), dim=-1) # Applying softmax
        attention_weights = self.dropout(attention_weights) # Applying dropout
        context_vector = (attention_weights @ values).transpose(1, 2) # Calculating context vector
        context_vector = context_vector.contiguous().view(b, num_tokens, self.d_out) # Reshaping context vector
        context_vector = self.out_proj(context_vector) # Applying output projection layer
        return context_vector
    

In [14]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self, x):
        mean = x.mean(dim = -1, keepdim=True)
        var = x.var(dim = -1, keepdim=True, unbiased=False)
        out_norm = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale*out_norm + self.shift

In [15]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg['emb_dim'], 4*cfg['emb_dim']),
            nn.GELU(),
            nn.Linear(4*cfg['emb_dim'], cfg['emb_dim']),
        )
    
    def forward(self, x):
        return self.layers(x)
        

In [16]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.attn = MultiHeadAttention(
            d_in = cfg['emb_dim'],
            d_out = cfg['emb_dim'],
            dropout = cfg['dropout_rate'],
            num_heads = cfg['num_heads'],
            qkv_bias = cfg['qkv_bias'],
            context_length = cfg['context_length']
        )
        self.FeedForward = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg['emb_dim'])
        self.norm2 = LayerNorm(cfg['emb_dim'])
        self.dropout = nn.Dropout(cfg['dropout_rate'])

    def forward(self,x):
        shortcut = x
        x = self.norm1(x)
        x = self.attn(x)
        x = self.dropout(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.FeedForward(x)
        x = self.dropout(x)
        x = x + shortcut

        return x

In [17]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['dropout_rate'])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )

        self.final_norm = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(cfg['emb_dim'], cfg['vocab_size'], bias = False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device = in_idx.device))

        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [18]:
GPT_CONFIG_124M = {
    'vocab_size': 50257,
    'emb_dim': 768,
    'context_length': 256,
    'n_layers': 12,
    'num_heads': 12,
    'dropout_rate': 0.1,
    'qkv_bias': False
}

In [19]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (attn): MultiHeadAttention(
        (w_key): Linear(in_features=768, out_features=768, bias=False)
        (w_value): Linear(in_features=768, out_features=768, bias=False)
        (w_query): Linear(in_features=768, out_features=768, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (FeedForward): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (attn): MultiHeadAttention(
        (w_key

In [20]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [21]:
## text_to_token_ids and token_ids_to_text
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special = {'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # adding the batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # removing the batch dimension
    return tokenizer.decode(flat.tolist())

In [22]:
# Example
start_content = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx = text_to_token_ids(start_content, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M['context_length']
)

print("Output text: \n", token_ids_to_text(token_ids, tokenizer))

Output text: 
 Every effort moves you 960esame WindsorFE Keith awaitedSer GaelListMine


### Calculating the text generation loss

In [51]:
text1 = "every effort moves"
text2 = "I really like"
target1 = " effort moves you"
target2 = " really like chocolate"

t1_tensor = text_to_token_ids(text1, tokenizer)
t2_tensor = text_to_token_ids(text2, tokenizer)

t3_tensor = text_to_token_ids(target1, tokenizer)
t4_tensor = text_to_token_ids(target2, tokenizer)

In [52]:
t1_tensor

tensor([[16833,  3626,  6100]])

In [53]:
t2_tensor

tensor([[  40, 1107,  588]])

In [54]:
t3_tensor

tensor([[3626, 6100,  345]])

In [55]:
t4_tensor

tensor([[ 1107,   588, 11311]])

In [35]:
inputs = torch.tensor([
    [16833,  3626,  6100],
    [40, 1107,  588]
])

In [56]:
targets = torch.tensor([
    [3626, 6100, 345],
    [588, 428, 11311]
])

In [57]:
with torch.no_grad():
    logits = model(inputs)
probas = torch.softmax(logits, dim=-1)
print(probas.shape)

torch.Size([2, 3, 50257])


In [58]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs: \n", token_ids)

Token IDs: 
 tensor([[[13207],
         [  552],
         [42826]],

        [[18236],
         [34817],
         [ 7055]]])


In [60]:
target_b1 = token_ids_to_text(targets[0], tokenizer)
tatget_b1_model = token_ids_to_text(token_ids[0].flatten(), tokenizer)

In [61]:
print(target_b1)
print(tatget_b1_model)

 effort moves you
hole compNetflix


In [62]:
text_idx = 0
target_probas_1 = probas[text_idx, [0,1,2], targets[text_idx]]

In [63]:
text_idx = 1
target_probas_2 = probas[text_idx, [0,1,2], targets[text_idx]]

In [64]:
print(target_probas_1, "\n", target_probas_2)

tensor([5.0549e-05, 2.7952e-05, 8.2801e-06]) 
 tensor([1.7212e-05, 1.5561e-05, 5.2184e-06])


In [65]:
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([ -9.8926, -10.4850, -11.7017, -10.9699, -11.0707, -12.1633])


In [66]:
ave_log_probas = torch.mean(log_probas)
print(ave_log_probas)

tensor(-11.0472)


In [68]:
neg_ave_log_probas = ave_log_probas * -1
print(neg_ave_log_probas) # this is called as cross entropy loss

tensor(11.0472)


In [69]:
print(logits.shape)
print(targets.shape)

torch.Size([2, 3, 50257])
torch.Size([2, 3])


In [70]:
logits = logits.flatten(0,1)
targets = targets.flatten()
print(logits.shape)
print(targets.shape)

torch.Size([6, 50257])
torch.Size([6])


In [71]:
loss = torch.nn.functional.cross_entropy(logits, targets)
print(loss)

tensor(11.0472)


### Perplexity
=> It is a measure often used alongside croos entropy loss to evaluate the performance of the models. It can provide a more interpretable way to understand the uncertainity of a model in predicting the next token in a sequence

=> Lower perplexity means the output predicted output is closer to the actual output.

=> perplexity = torch.exp(loss) ex:- it gives 47678 that means the model is not sure which one of the 47678 tokens in the vocab to generate as the next token

## Preparing the dataset

In [75]:
file_path = "/Users/siddartha/Desktop/github/llms-from-scratch/Chapter_2_Working_with_text_data/the_verdict.txt"
with open(file_path, "r", encoding="utf-8") as f:
    text_data = f.read()

In [78]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Total characters: \n", total_characters)
print("Total tokens: \n", total_tokens)

Total characters: 
 20479
Total tokens: 
 5145


In [80]:
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

In [84]:
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDatasetv1(Dataset):
    def __init__(self, text, stride, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunks = token_ids[i:i+max_length]
            target_chunks = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunks))
            self.target_ids.append(torch.tensor(target_chunks))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]  

In [85]:
def create_dataloader_v1(text, batch_size, max_length, stride, shuffle=False, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetv1(text=text, max_length=max_length, stride=stride, tokenizer=tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

In [86]:
torch.manual_seed(123)

train_loader = create_dataloader_v1(
    text=train_data, 
    batch_size=2, 
    max_length = GPT_CONFIG_124M["context_length"], 
    stride = GPT_CONFIG_124M['context_length'],
    drop_last=True,
    shuffle = True,
)

val_loader = create_dataloader_v1(
    text = val_data,
    batch_size=2,
    max_length = GPT_CONFIG_124M['context_length'],
    stride=GPT_CONFIG_124M['context_length'],
    drop_last = True,
    shuffle = True,
)

In [88]:
print("Train Loader: ")
for x,y in train_loader:
    print(x.shape, y.shape)

Train Loader: 
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])


In [89]:
print("Validation Loader: ")
for x, y in val_loader:
    print(x.shape, y.shape)

Validation Loader: 
torch.Size([2, 256]) torch.Size([2, 256])


In [91]:
print(len(train_loader))

9


In [92]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(
        logits.flatten(0,1), target_batch.flatten()
    )
    return loss

In [93]:
## Function to compute the training and validation loss
def calc_loss_loader(data_loader, model, device, num_batches = None):
    total_loss = 0
    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss+=loss.item()
        else:
            break
    return total_loss / num_batches

In [95]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
train_loss = calc_loss_loader(train_loader, model, device)
val_loss = calc_loss_loader(val_loader, model, device)
print("Training Loss: \n", train_loss)
print("Validation Loss: \n", val_loss)

Training Loss: 
 10.997160805596245
Validation Loss: 
 10.98913860321045
